In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from cvt_utils import SpaceIterMesh, plot_cell_size_and_density, plot_seeds_and_cells, plot_bound_paths, display_animation, plot_computed_cell_size

plt.style.use("dark_background")
np.random.seed(21)

# Problem definition
x_min = 0.0
x_max = 1.0
# n_cells = 50
n_cells = 100
# n_iters = 300
# n_iters = 2000
# n_iters = 5000
# n_iters = 7000
n_iters = 10_000

mesh = SpaceIterMesh(x_min, x_max, n_cells, n_iters)

# Cell size function
# mesh.cell_size = lambda x: 1.0 + 0.0 * x
# mesh.cell_size = lambda x: np.exp(2*x)
# mesh.cell_size = lambda x: 0.51 + 0.5 * np.sin(2 * np.pi * x)
mesh.cell_size = lambda x: 0.5025 + 0.5 * np.sin(2 * np.pi * x)

In [ ]:
# Print cell size statistics
x = np.linspace(x_min, x_max, 1000)
assert mesh.cell_size is not None, "Cell size function not set"
print("Cell size stats:")
print(f"min: {np.min(mesh.cell_size(x)):.4f}")
print(f"max: {np.max(mesh.cell_size(x)):.4f}") # Should be 0.505 
print(f"mean: {np.mean(mesh.cell_size(x)):.3f}")
print(f"std: {np.std(mesh.cell_size(x)):.3f}")

# Print cell density statistics
print()
print(f"Cell density min: {np.min(mesh.cell_density(x)):.3f}")
print(f"Cell density max: {np.max(mesh.cell_density(x)):.3f}")
print(f"Cell density mean: {np.mean(mesh.cell_density(x)):.3f}")
print(f"Cell density std: {np.std(mesh.cell_density(x)):.3f}")

# Plot cell size and density
plot_cell_size_and_density(mesh)

### Iterations

In [ ]:
# Generate random x values for the first iteration, sort them, and compute cell boundaries
mesh.seed_matrix[:, 0] = np.sort(np.random.uniform(x_min, x_max, n_cells))

mesh.update_cell_bounds()
   
for iter in range(n_iters):
    mesh.update_cell_bounds(iter)
    if iter < n_iters - 1:
        mesh.update_cell_seeds(iter)

In [ ]:
html = display_animation(mesh, duration=3.0, fps=60)
html

In [ ]:
plot_bound_paths(mesh, 0, n_iters - 1)

In [ ]:
# Compute first derivative of the seed positions over iterations (dx/di = dx)
dseed_matrix = np.diff(mesh.seed_matrix, axis=1)

# Plot the min, max, RMS, and std of the first derivative of the seed positions over iterations
min_dseed = np.min(np.abs(dseed_matrix), axis=0)
max_dseed = np.max(np.abs(dseed_matrix), axis=0)
rms_dseed = np.sqrt(np.mean(dseed_matrix**2, axis=0))

plt.figure(figsize=(10, 6))
plt.plot(min_dseed, label="Min", linewidth=1, color="green")
plt.plot(max_dseed, label="Max", linewidth=1, color="red")
plt.plot(rms_dseed, label="RMS", linewidth=1, color="cyan")
plt.title("Convergence of first derivatives")
plt.xlabel("Iteration")
plt.ylabel("Seed Position Change")
plt.yscale("log")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Compute second derivative of the seed positions over iterations
d2seed_matrix = np.diff(dseed_matrix, axis=1)

# Plot the min, max, and RMS of the second derivative of the seed positions over iterations
min_d2seed = np.min(np.abs(d2seed_matrix), axis=0)
max_d2seed = np.max(np.abs(d2seed_matrix), axis=0)
rms_d2seed = np.sqrt(np.mean(d2seed_matrix**2, axis=0))

plt.figure(figsize=(10, 6))
plt.plot(min_d2seed, label="Min", linewidth=1, color="green")
plt.plot(max_d2seed, label="Max", linewidth=1, color="red")
plt.plot(rms_d2seed, label="RMS", linewidth=1, color="cyan")
plt.title("Convergence of second derivatives")
plt.xlabel("Iteration")
plt.ylabel("Change of Seed Position Change")
plt.yscale("log")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Compute third derivative of the seed positions over iterations
d3seed_matrix = np.diff(d2seed_matrix, axis=1)

# Plot the min, max, and RMS of the third derivative of the seed positions over iterations
min_d3seed = np.min(np.abs(d3seed_matrix), axis=0)
max_d3seed = np.max(np.abs(d3seed_matrix), axis=0)
rms_d3seed = np.sqrt(np.mean(d3seed_matrix**2, axis=0))

plt.figure(figsize=(10, 6))
plt.plot(min_d3seed, label="Min", linewidth=1, color="green")
plt.plot(max_d3seed, label="Max", linewidth=1, color="red")
plt.plot(rms_d3seed, label="RMS", linewidth=1, color="cyan")
plt.title("Convergence of third derivatives")
plt.xlabel("Iteration")
plt.ylabel("Change of Change of Seed Position Change")
plt.yscale("log")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
plot_computed_cell_size(mesh)

### Energy

In [ ]:
# Compute total energy per iteration on a log scale
total_energy = mesh.total_energy()

plt.figure(figsize=(10, 6))
plt.plot(np.sqrt(total_energy), label="Total Energy", linewidth=1, color="cyan")
# plt.plot(np.log10(total_energy), label="Total Energy", linewidth=1, color="cyan") total_energy, label="Total Energy", linewidth=1, color="cyan")
plt.title("Total Energy Over Iterations")
plt.xlabel("Iteration")
plt.ylabel("Total Energy")
plt.legend()
plt.yscale("log")
plt.grid(True, alpha=0.3)
plt.show()